In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')
sys.path.append(work_dir)


In [2]:
from src.model.models import CardioTrain
from src.database.dbconnection import getconnection
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from transform.TransformData import DataTransform

In [3]:
engine = getconnection()
Session = sessionmaker(bind=engine)
session = Session()

Conected successfully to database postgres!


In [4]:
try:
    if inspect(engine).has_table('CardioTrain'):
        CardioTrain.__table__.drop(engine)
    CardioTrain.__table__.create(engine)
    print("Table created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
finally:
    engine.dispose()

Table created successfully.


In [5]:
try:
    file = DataTransform('../data/cardio_train.csv')
    file.df.to_sql('CardioTrain', con=engine, if_exists='append', index=False)
    print("Data uploaded")

except SQLAlchemyError as e:
    print(f"Database error: {e}")

except Exception as e:
    print(f"Error: {e}")

finally:
    if hasattr(engine, 'dispose'):
        engine.dispose()

    if 'session' in locals():
        session.close()

Data uploaded
